In [321]:
#aim is to build optimal portfolio programme that uses MPT mean convergence to make perfect weighted portfolio of given stocks
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from scipy.optimize import minimize
nse_stocks = [
    'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS', 'ICICIBANK.NS',
    'HINDUNILVR.NS', 'ITC.NS', 'KOTAKBANK.NS', 'LT.NS', 'SBIN.NS'
]
#NSE = yf.download("^NSEI")
start_date = "2021-01-01"
end_date = end_date = datetime.today().strftime('%Y-%m-%d')

returns = pd.DataFrame()
df_close = pd.DataFrame()

for ticker in nse_stocks:
     data = yf.download(ticker, start=start_date, end=end_date)
     df_close[ticker] = data["Close"]
    
returns = df_close.pct_change().interpolate()
df_ret = returns.to_csv("stock_closing_prices.csv")
#calculating covariance
covariance = returns.cov()
covariance.to_csv("stock_covariance.csv")

n = len(nse_stocks)
stock_return_list = []
returns.to_csv("stock_closing_prices.csv")
for i in returns.columns:
    avg = returns[i].mean()
    stock_return_list.append(avg)

stock_return  = np.array(stock_return_list)

cov_mat = pd.read_csv("stock_covariance.csv",index_col = 0)

riskfree_rate = .06/252

def SR (weights, stock_return, cov_mat, riskfree_rate):
    PF_return = np.dot(weights, stock_return)
    var = np.dot(weights.T, np.dot(cov_mat, weights))
    sharpe = (PF_return-riskfree_rate)/np.sqrt(var)
    return -sharpe
    return PF_return

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, .6) for i in range(n))

init_guess = np.array([1/n]*n)
optimized = minimize(
    SR, init_guess,
    args=(stock_return, cov_mat.values, riskfree_rate),
    method='SLSQP', bounds=bounds, constraints=constraints
)
opt_w = optimized.x
for i,j in zip(nse_stocks,opt_w):
    print(f"{i}: {j*100}%")

final_opt_ret = np.dot(opt_w,stock_return) 
ann_ret = final_opt_ret*252
final_opt_STDEV = np.sqrt(np.dot(opt_w.T,np.dot(cov_mat,opt_w)))
ann_std = final_opt_STDEV*np.sqrt(252)
sharpe_max = ( ann_ret- (riskfree_rate)*252) / ann_std
print(f"Sharpe max: {sharpe_max}")
print(f"est return:  {ann_ret*100}%")
print(f"Optimal std: {ann_std*100}%")





[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

RELIANCE.NS: 0.0%
TCS.NS: 0.0%
INFY.NS: 0.0%
HDFCBANK.NS: 2.288512939686404e-15%
ICICIBANK.NS: 28.2529834892829%
HINDUNILVR.NS: 4.868068108586894e-16%
ITC.NS: 31.33006127208528%
KOTAKBANK.NS: 2.2925474375340104e-15%
LT.NS: 26.57551665754509%
SBIN.NS: 13.84143858108672%
Sharpe max: 1.165791564101658
est return:  26.260874229799242%
Optimal std: 17.37949977825751%


10


0.008861539454294083
